## Download from Cepik api for motoliczby.pl

Dokumnetacja APi Cepik: https://api.cepik.gov.pl/doc
Filtrowanie: filter[<nazwa atrybutu>]=<szukana wartość>
    
    

In [1]:
import pandas as pd
import numpy as np
import json
import time
import requests
import os
import math
from datetime import datetime

In [2]:
# names of Wojewodztwa
wojew_names = {
"02" : "DOLNOŚLĄSKIE",
"04" : "KUJAWSKO-POMORSKIE",
"06" : "LUBELSKIE",
"08" : "LUBUSKIE",
"10" : "ŁÓDZKIE",
"12" : "MAŁOPOLSKIE",
"14" : "MAZOWIECKIE",
"16" : "OPOLSKIE",
"18" : "PODKARPACKIE",
"20" : "PODLASKIE",
"22" : "POMORSKIE",
"24" : "ŚLĄSKIE",
"26" : "ŚWIĘTOKRZYSKIE",
"28" : "WARMIŃSKO-MAZURSKIE",
"30" : "WIELKOPOLSKIE",
"32" : "ZACHODNIOPOMORSKIE",
"XX" : "NIEOKREŚLONE"
}




In [3]:
# Set up CEPIK API parameters
#wojew = ["02","04","06","08","10","12","14","16","18","20","22","24","26","28","30","32","XX"] # Wojewodztwa backup 

wojew = ["02","04","06","08","10","12","14","16","18","20","22","24","26","28","30","32","XX"] 
years = ["2024"]
MDStart = "0101" # mont-data start  0101 as 01 JAN
MDEnd = "1231" # mont-data end 1231 as 31 DEC



In [4]:
# Settings folders

main_data_path = "D:/Dokumenty/Tomek/Data_science/Cepik/nowe_samochody" # please define here a upload path.
current_dic = "/motoliczby" # Define a folder
now_file = "Api_download_" + datetime.now().strftime("%Y_%m_%d_%H_%M_%S") # Donwload file name pattern 
upload_path = main_data_path + current_dic + "_" + now_file

base_name_xls = "Cepik_" # base name of the xls/csv file
path_to_merge = upload_path + current_dic + "all/" # name patern of sumerised date file
all_merged_data_file_name ="Cepik_all_data_in_one_file.csv" # Name of the file with all data
save_total_of_year = False # collect and save all data within a year. It may contain lot of records, slow down the process or make an error


In [5]:
# url and filters

based_url = "https://api.cepik.gov.pl/pojazdy?&limit=500" 
filtr = '&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=false'

#Backuped filters
#filtr = '&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=true'
#filtr = '&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=false&filter[rodzaj-pojazdu]=Motocykl'
#filtr = '&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=false&filter[rodzaj-pojazdu]=Motocykl&filter[marka]=bmw&filter[model]=K1'
#filtr = '&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=false&filter[rodzaj-pojazdu]=Motocykl'

#filtr = '&filter[pochodzenie-pojazdu]=NOWY+&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true'
#filtr = '&filter[pochodzenie-pojazdu]=NOWY&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true'
#filtr = '&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true'
#filtr = '&filter[pochodzenie-pojazdu]=NOWY+ZAKUPIONY+W+KRAJU&filter[rodzaj-pojazdu]=SAMOCHÓD+OSOBOWY&pokaz-wszystkie-pola=true&sort=marka&sort=model&sort=typ&sort=wariant'
#filtr = '&filter[pochodzenie-pojazdu]=NOWY+ZAKUPIONY+W+KRAJU&filter[pochodzenie-pojazdu]=NOWY+IMPORT+INDYW&filter[pochodzenie-pojazdu]=NOWY+ZAKUPIONY+W+KRAJU&pokaz-wszystkie-pola=true&sort=id'
#filtr = '&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=false'

#elektryczne
#filtr = '&pokaz-wszystkie-pola=true&sort=-id&tylko-zarejestrowane=false&filter[rodzaj-paliwa]=ENERGIA+ELEKTRYCZNA'
#filtr ="&filter[rodzaj-pojazdu]=SAMOCHÓD+OSOBOWY"
#filtr = '&filter[marka]=TOYOTA&filter[pochodzenie-pojazdu]=NOWY+ZAKUPIONY+W+KRAJU&filter[pochodzenie-pojazdu]=NOWY+IMPORT+INDYW&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true'


In [6]:
# data frames declaration
all_cars_in_years = pd.DataFrame()
cars_in_years_and_woj = pd.DataFrame()

try:
    del cars, df
except Exception:
    print("no df to del")

no df to del


In [ ]:
# main Cell

# years loop
for y in years:
#for w in wojew:

    print ("-"*40)
    print ("Scope of data capture (years): start:", years[0] ,"end:", years[-1] )
    print("Year:", y)
    
    # Wojewodztwo loop
    for w in wojew:
                    
        # variables
        wojewodz = "&wojewodztwo=" + w
        period = "&data-od=" + y + MDStart+ "&data-do=" + y + MDEnd
        path_to_save = upload_path + "/" + y + "/" + w + "/" 
        path_to_save_all = upload_path + "/" + y + "/" 
        
        rec_no = 0
        step = 1
        finished = False 
        number_of_waits = 0
        page = "&page=1" 
        
    
        #prints 
        print ("-"*20)
        print("Woj:", w," ",wojew_names[w], "/",y)
        
               
        # define first page
        url = based_url + period +  filtr + wojewodz + page
        
        # scrapping loop within the Woj
        while not finished:
            
            print ("We dig in:", url)

            #------------------------------------    
            data_temp = requests.get(url) 
            #------------------------------------
            
            # check the respond code if no errors
            if data_temp.status_code != 200: # 200 is correct answer (no errors)
                number_of_waits = number_of_waits + 1
                print ("digging status NOK: ", data_temp.status_code, "wait for 5 sek.", number_of_waits , "times")
                tc_retry = True
                time.sleep(5) # Wait for 5 sek (I know this doesne't help much)
                
            else:
                tc_retry = False
                
                #print ("digging OK -", "data statatus code:",data_temp.status_code)
                data = data_temp.json()
                df = pd.json_normalize(data, record_path =['data'])

                # how many times we need to iterate to download all Wojewodztwo
                tc_links = pd.json_normalize(data)
                                  
                # numbers of records
                rec_no = tc_links["meta.count"][0]
                records_tc =  rec_no
                rec_no = math.ceil(rec_no/500)
                                
                # Stop the loop if there is no records to downlaod
                if records_tc == 0:
                    print ("----NO RECORDS DO DOWNLOAD","page/step: ",step,"/", rec_no," Records in woj: ",w, "records to download:",records_tc ," pages remaining: ", rec_no - step)
                    break;
                                
                # Make a directory if thera are records to save
                if records_tc != 0 and step == 1:
                    try: 
                        os.makedirs (path_to_save)
                        print (path_to_save, " successfully created")
                    except Exception:
                        print("dic: ", path_to_save, "aready exist" )

                
                # Define next url to capture
                try: 
                    next_url = tc_links["links.next"][0]
                except Exception:
                    next_url = "No next url - that was the last one"
                    
                # define next url to download    
                last_url = tc_links["links.last"][0]
                              
                # ----------- append data into dataframes of woj ----------------
                cars_in_years_and_woj = pd.concat([cars_in_years_and_woj, df])
                
                # Printouts
                print(f">>page/step: {step}/{rec_no} Records in woj: {w} records to download: {records_tc} total rec: {len(cars_in_years_and_woj)} Unique: {cars_in_years_and_woj['id'].nunique()} pages remaining: {rec_no - step}")
                
                # save each set of data from particular step steps into the csv
                now_file = "generated_" + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
                #name_xls = path_to_save + base_name_xls + y + "_woj_" +w + "_step_" + str(step) + "-" + now_file + ".csv"
                #name_xls_total = path_to_save_all + base_name_xls + y + "_woj_" +w + "_total_" + "-" + now_file + ".csv"
                name_xls = f"{path_to_save}{base_name_xls}{y}_woj_{w}_step_{step}-{now_file}.csv"
                name_xls_total = f"{path_to_save_all}{base_name_xls}{y}_woj_{w}_total_-{now_file}.csv"

                
                df.to_csv(path_or_buf = name_xls,
                            index = False,
                            sep = ";",
                            encoding = 'utf-8')
             

                # end of wojewodztwo
                if last_url == url or next_url == "No next url - it was last one":
                    
                    
                    finished = True

                    # Save df                 
                    cars_in_years_and_woj.to_csv(
                                        path_or_buf= name_xls_total,
                                        index = False,
                                        sep = ";",
                                        encoding = 'utf-8')
                    
                    # printouts end of the wojewodztwo
                    print("*" * 60)
                    print(f"End of Województwo. Województwo {w}/{y} is downloaded")
                    print(f"File saved: total rec: {len(cars_in_years_and_woj)} Unique: {cars_in_years_and_woj['id'].nunique()}")
                    print(f">>>>>>>> Duplicates found: {len(cars_in_years_and_woj) - cars_in_years_and_woj['id'].nunique()}")
                    print("*" * 60)


                    if save_total_of_year == True:
                        # ----------- append data into dataframes of year ----------------
                        #all_cars_in_years = all_cars_in_years.append(cars_in_years_and_woj)
                        all_cars_in_years = pd.concat([all_cars_in_years , cars_in_years_and_woj])

                    # Remove all data from df
                    cars_in_years_and_woj = cars_in_years_and_woj.iloc[0:0]
  
                else:    
                    step = step + 1
                    url = next_url
                print ("-"*5)
  
    if save_total_of_year == True:
    
        # Save all data within a year   
        if len(cars_in_years_and_woj) > 0:

            # Verification of duplicates
            cars_in_years_and_woj_records = len(cars_in_years_and_woj)
            cars_in_years_and_woj_unique = cars_in_years_and_woj['id'].nunique()

            cars_in_years_and_woj.to_csv(
                path_or_buf= path_to_save_all + base_name_xls + y + "_all_woj_total_" + "-" + now_file + ".csv",
                index = False,
                sep = ";",
                encoding = 'utf-8')


            # Printouts end of the year
            
            print("-------------------------------------")
            print(f"------ Year {y} Successfully downloaded to {path_to_save_all}")
            print(f"Total records: {cars_in_years_and_woj_records}")
            print(f"Total unique: {cars_in_years_and_woj_unique}")

            if cars_in_years_and_woj_records == cars_in_years_and_woj_unique:
                print("OK, no duplicates found")
            else:
                print("!!!! Possible duplicates, please check it!")

            print("-------------------------------------")

            
        else:
            print("No records found in: ",y," year")

        # end of Year loop

In [ ]:
print ("koniec")